In [ ]:
import pandas as pd

In [ ]:
# Charger un fichier JSON en DataFrame
df = pd.read_json("../exchange_data/raw_taux_change.json") 

In [ ]:
df['taux_achat'] = df['taux_achat'].str.replace(',', '.').astype(float)
df['taux_vente'] = df['taux_vente'].str.replace(',', '.').astype(float)

In [ ]:
# Extraction de la quantité et du nom de devise
df[['unite_montant', 'nom_devise']] = df['devise_complete'].str.extract(r'(\d+)\s+(.*)')
df['unite_montant'] = df['unite_montant'].astype(int)

In [ ]:
# Dictionnaire des abréviations ISO ou personnalisées
abr_dict = {
    "EURO": "EUR",
    "DOLLAR U.S.A.": "USD",
    "DOLLAR CANADIEN": "CAD",
    "LIVRE STERLING": "GBP",
    "LIVRE GIBRALTAR": "GIP",
    "FRANC SUISSE": "CHF",
    "RIYAL SAOUDIEN": "SAR",
    "DINAR KOWEITIEN": "KWD",
    "DIRHAM E.A.U.": "AED",
    "RIYAL QATARI": "QAR",
    "DINAR BAHREINI": "BHD",
    "YENS JAPONAIS": "JPY",
    "RIYAL OMANI": "OMR"
}

# Création de la colonne abr_devise
df['abr_devise'] = df['nom_devise'].map(abr_dict)

In [ ]:
df['taux_achat'] = df['taux_achat']/df['unite_montant']
df['taux_vente'] = df['taux_vente']/df['unite_montant']

In [ ]:
df.drop(["devise_complete","unite_montant"], axis=1, inplace=True)

In [ ]:
abr_devises = ['AED', 'BHD', 'CAD', 'CHF', 'EUR', 'GBP', 'GIP', 'JPY', 'KWD','OMR', 'QAR', 'SAR', 'USD']

In [ ]:
df_combined = pd.DataFrame()
for abr_devise in abr_devises:
    sub_df=df[df["abr_devise"]==abr_devise]
    sub_df=sub_df.sort_values(by='date')
    sub_df=sub_df.set_index('date')
    sub_df=sub_df.interpolate(method='time', limit_direction='both')
    df_combined = pd.concat([df_combined, sub_df])

In [ ]:
df_combined['taux_moyen'] = (df_combined['taux_achat']+df_combined['taux_vente'])/2
df_combined['ecart']      = df_combined['taux_vente'] - df_combined['taux_achat']

In [ ]:
df_combined = df_combined.reset_index()
df_combined['id'] = range(1, len(df_combined) + 1)

In [ ]:
df_combined=df_combined[['id', 'date', 'nom_devise', 'abr_devise', 'taux_achat', 'taux_vente','ecart', 'taux_moyen']]

In [ ]:
df_combined.to_json("../exchange_data/clean_taux_change.json", orient="records", force_ascii=False, indent=2)